# カーネルを使った強化学習

参考：
* [Kernel-Based Reinforcement Learning: A Finite-Time Analysis](https://arxiv.org/abs/2004.05599)

状態行動空間が連続のMDPではどのようにRLすればよいでしょうか？
今回はカーネルを使った強化学習を見てみます．

表記：
* $(x, a)$に定義された測度$P$と任意の関数$f$について，$P f(x, a)=P(\cdot \mid x, a) f=\int f(y) \mathrm{d} P(y \mid x, a)$
* $\mathcal{X}$と$\mathcal{A}$上に定義された測度：$\rho:(\mathcal{X} \times \mathcal{A})^2 \rightarrow \mathbb{R}_{\geq 0}$
* $\mathcal{X}$上の測度空間：$\left(\mathcal{X}, \mathcal{T}_{\mathcal{X}}\right)$．$\mathcal{T}_{\mathcal{X}}$はσ代数です．
* 報酬関数：$\left\{r_h\right\}_{h \in[H]}$は$\mathcal{X} \times \mathcal{A}$ to $[0,1]$の集合
* 遷移カーネル：$P=\left\{P_h\right\}_{h \in[H]}$．ここで，$P_h$は$(\mathcal{X} \times \mathcal{A}) \times \mathcal{T}_{\mathcal{X}}$ to $\mathbb{R}_{\geq 0}$なるカーネル．
* ベルマン方程式：$Q_h^*(x, a) \stackrel{\text { def }}{=} r_h(x, a)+\int_{\mathcal{X}} V_{h+1}^*(y) \mathrm{d} P_h(y \mid x, a)$
* リグレット：$\mathcal{R}(K) \stackrel{\text { def }}{=} \sum_{k=1}^K\left(V_1^*\left(x_1^k\right)-V_1^{\pi_k}\left(x_1^k\right)\right)$
* $\mathcal{F}_h^k$を確率変数$\left\{x_h^s, a_h^s, x_{h+1}^s, r_h^s\right\}_{s=1}^{k-1} \cup\left\{x_{h^{\prime}}^k, a_{h^{\prime}}^k, x_{h^{\prime}+1}^k, r_{h^{\prime}}^k\right\}_{h^{\prime}<h}$によって生成されるσ代数とします．
* $\left(\mathcal{F}_h^k\right)_{k, h}$をそのフィルトレーションとします．
* 簡単のために報酬は既知とします．

仮定：

---

* $(\mathcal{X}\times \mathcal{A})\times (\mathcal{X}\times \mathcal{A})$上の測度$\rho$は既知であるとする．
* 任意の$\left(x, x^{\prime}, a, a^{\prime}\right)$について，
$$
\rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]=\rho_{\mathcal{X}}\left(x, x^{\prime}\right)+\rho_{\mathcal{A}}\left(a, a^{\prime}\right)
$$
が成り立つような測度$\rho_{\mathcal{X}}$と$\rho_{\mathcal{A}}$が存在する．

---

報酬と遷移は次の意味でリプシッツ連続です

$$
\left|r_h(x, a)-r_h\left(x^{\prime}, a^{\prime}\right)\right| \leq \lambda_r \rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]
$$

および

$$
W_1\left(P_h(\cdot \mid x, a), P_h\left(\cdot \mid x^{\prime}, a^{\prime}\right)\right) \leq \lambda_p \rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]
$$

ここで，$W_1$は1-Wasserstein距離です．つまり，$W_1(\mu, \nu) \stackrel{\text { def }}{=}\sup _{f: \operatorname{Lip}(f) \leq 1} \int_{\mathcal{X}} f(y)(\mathrm{d} \mu(y)-\mathrm{d} \nu(y))$

---

**定義：測度空間の被覆**

$(\mathcal{U}, \rho)$を測度空間とします．任意の$u\in \mathcal{U}$について，$\mathcal{B}(u, \sigma)=\{v \in \mathcal{U}: \rho(u, v) \leq \sigma\}$とします．
$C_\sigma \subset \mathcal{U}$は$\mathcal{U} \subset \bigcup_{u \in \mathcal{C}_\sigma} \mathcal{B}(u, \sigma)$を満たすときに$(\mathcal{U}, \rho)$の$\sigma$-coveringであるといいます．
また，$(\mathcal{U}, \rho)$の$\sigma$-covering numberを

$$\mathcal{N}(\sigma, \mathcal{U}, \rho) \stackrel{\text { def }}{=} \min \left\{\left|\mathcal{C}_\sigma\right|: \mathcal{C}_\sigma \text{ is a $\sigma$-covering of } (\mathcal{U}, \rho)\right\}$$

として定義します．

## カーネルUCBVI

$u, v \in \mathcal{X} \times \mathcal{A}$とします．関数$g: \mathbb{R}_{\geq 0} \rightarrow[0,1]$について，カーネル関数を

$$\psi_\sigma(u, v) \stackrel{\text { def }}{=} g(\rho[u, v] / \sigma)$$

として定めます．

---

**仮定**

関数$g: \mathbb{R}_{\geq 0} \rightarrow[0,1]$は微分可能，non-increasing，$g(4) > 0$を満たし，また，次を満たす定数$C^g_1, C^g_2$が存在します：

$$
g(z) \leq C_1^g \exp \left(-z^2 / 2\right) \text { and } \sup _z\left|g^{\prime}(z)\right| \leq C_2^g
$$

例えば$g(z)=\exp \left(-z^2 / 2\right)$のようなガウスカーネルが仮定を満たしています．

---

カーネルUCBVIではカーネル関数を使って過去のデータを重み付けし，報酬や遷移確率を推定します．

$(x, a)$ and $(s, h) \in[K] \times[H]$に対して，重み関数を

$$
\begin{aligned}
& w_h^s(x, a) \stackrel{\text { def }}{=} \psi_\sigma\left((x, a),\left(x_h^s, a_h^s\right)\right) \quad \text { and } \\
& \widetilde{w}_h^s(x, a) \stackrel{\text { def }}{=} \frac{w_h^s(x, a)}{\beta+\sum_{l=1}^{k-1} w_h^l(x, a)},
\end{aligned}
$$

とします．ここで$\beta > 0$は正則化項です．この重みを使って，

$$
\widehat{P}_h^k(y \mid x, a) \stackrel{\text { def }}{=} \sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) \delta_{x_{h+1}^s}(y) .
$$

のように遷移を計算します．
そして，推定した遷移を使って

$$
\widetilde{Q}_h^k(x, a)={r}_h(x, a)+\widehat{P}_h^k V_{h+1}^k(x, a)+\mathrm{B}_h^k(x, a)
$$

のようにQ値を計算します．ここで$B$は探索用のボーナスです．
これを使って，

$$
Q_h^k(x, a) \stackrel{\text { def }}{=} \min _{s \in[k-1]}\left(\widetilde{Q}_h^k\left(x_h^s, a_h^s\right)+L_h \rho\left[(x, a),\left(x_h^s, a_h^s\right)\right]\right)
$$

のようにしてQ値の間隔を補完します．

---

**ボーナスについて**


$$
\mathbf{C}_h^k(x, a) \stackrel{\text { def }}{=} \beta+\sum_{s=1}^{k-1} w_h^s(x, a)
$$

を訪問回数の一般化とします．これを使って，ボーナスを

$$
\mathrm{B}_h^k(x, a) \approx \frac{H}{\sqrt{\mathbf{C}_h^k(x, a)}}+\frac{\beta H}{\mathbf{C}_h^k(x, a)}+L_1 \sigma
$$

として定義します．

正確に書くと，
$$
\begin{aligned}
\mathrm{B}_h^k(x, a)
& ={\left(\sqrt{\frac{H^2 \mathbf{v}_{\mathrm{p}}(k, \delta / 6)}{\mathbf{C}_h^k(x, a)}}+\frac{\beta H}{\mathbf{C}_h^k(x, a)}+\mathbf{b}_{\mathrm{p}}(k, \delta / 6) \sigma\right)}
\end{aligned}
$$

であり，

$$
\begin{aligned}
& \mathbf{v}_{\mathrm{p}}(k, \delta)=\widetilde{\mathcal{O}}\left(d_1\right)=2 \log \left(H \mathcal{N}\left(\sigma^2 /(K H), \mathcal{X} \times \mathcal{A}, \rho\right) \frac{\sqrt{1+k / \beta}}{\delta}\right) \\
& \mathbf{b}_{\mathrm{p}}(k, \delta)=\widetilde{\mathcal{O}}\left(L_1+\sqrt{d_1}\right)=\frac{4 C_2^g}{\beta}+\sqrt{\mathbf{v}_{\mathrm{p}}(k, \delta)} \frac{C_2^g}{\beta^{3 / 2}}+2 \lambda_p L_1\left(1+\sqrt{\log ^{+}\left(C_1^g k / \beta\right)}\right)
\end{aligned}
$$

です．（$d_1$は$(\mathcal{X}\times \mathcal{A}, \rho)$の被覆次元）

以上をまとめて，Kernel-UCBVIは次の処理を行います：

![KUCBVI](figs/Kernel-UCBVI-OptimisticQ.png)
![KUCBVI](figs/Kernel-UCBVI.png)

## リグレット解析


このアルゴリズムについて，次のリグレットバウンドが成立します：

$$
\mathcal{R}(K) \leq \widetilde{\mathcal{O}}\left(H^2 \sqrt{\left|\mathcal{C}_\sigma\right| K}+L_1 K H \sigma+H^3\left|\mathcal{C}_\sigma\right|\left|\widetilde{\mathcal{C}}_\sigma\right|\right)
$$

ここで，$\left|\mathcal{C}_\sigma\right|$ と $\left|\widetilde{\mathcal{C}}_\sigma\right|$は$\left(\mathcal{X}\times \mathcal{A}, \rho\right)$と$\left(\mathcal{X}, \rho_{\mathcal{X}}\right)$の$\sigma$-coveringです．
また，$L_1$は最適Q値のリプシッツ定数です．

### ステップ１：遷移の集中不等式

証明の流れは基本的にUCBVIと同じで，まずは遷移の集中不等式を出します．

次が成立します：

任意の$(k, h, x, a)$について，

$$
\left|\widehat{P}_h^k V_{h+1}^*(x, a)-P_h V_{h+1}^*(x, a)\right| \leq \sqrt{\frac{H^2 \mathbf{v}_{\mathrm{p}}(k, \delta)}{\mathbf{C}_h^k(x, a)}}+\frac{\beta H}{\mathbf{C}_h^k(x, a)}+\mathbf{b}_{\mathrm{p}}(k, \delta) \sigma
$$

**証明**

$(x, a, h)$を固定します．$V=V^*_{h+1}$とします．このとき，

$$
\begin{aligned}
\left|\widehat{P}_h^k V(x, a)-P_h V(x, a)\right| & \leq\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V\left(x_{h+1}^s\right)-P_h V(x, a)\right)\right|+\left|\frac{\beta P_h V(x, a)}{\mathbf{C}_h^k(x, a)}\right| \\
& \leq\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V\left(x_{h+1}^s\right)-P_h V(x, a)\right)\right|+\frac{\beta H}{\mathbf{C}_h^k(x, a)}
\end{aligned}
$$

が成り立ちます．また，仮定より$V$は$L_1$-Lipshitzなので，

$$
\begin{aligned}
& \left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V\left(x_{h+1}^s\right)-P_h V(x, a)\right)\right| \\
& \leq\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V\left(x_{h+1}^s\right)-P_h V\left(x_h^s, a_h^s\right)\right)\right|+\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(P_h V\left(x_h^s, a_h^s\right)-P_h V(x, a)\right)\right| \\
& \leq\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V\left(x_{h+1}^s\right)-P_h V\left(x_h^s, a_h^s\right)\right)\right|+L_1\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) W_1\left(P_h\left(\cdot \mid x_h^s, a_h^s\right), P_h(\cdot \mid x, a)\right)\right| \\
& \leq\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V\left(x_{h+1}^s\right)-P_h V\left(x_h^s, a_h^s\right)\right)\right|+\lambda_p L_1\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) \rho\left[\left(x_h^s, a_h^s\right),(x, a)\right]\right| \\
& \leq\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V\left(x_{h+1}^s\right)-P_h V\left(x_h^s, a_h^s\right)\right)\right|+\lambda_p L_1 2 \sigma\left(1+\sqrt{\log ^{+}\left(C_1^g k / \beta\right)}\right)
\end{aligned}
$$

が成り立ちます．
最後の行では[RL_useful_lemma.ipynb](RL_useful_lemma.ipynb)の
**補題：重み付きの非負実数の和をバウンドするときに便利**を使いました．

続いて，$W_s \stackrel{\text { def }}{=} V\left(x_{h+1}^s\right)-P_h V\left(x_h^s, a_h^s\right)$として，この一項目である$\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) W_s\right|$をバウンドしていきます．
$(W_s)_s$は$(\mathcal{F}_h^s)_s$についてのマルチンゲール差分列なのでHoeffdingが使えそうですが，これは重み付きの和であり，重み自体も確率変数です．よって，特殊なHoeffdingが必要になります．

詳細は割愛しますが，論文のLemma 2を使うと，（TODO: Hoeffingの詳細）

$$
\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) W_s\right| \leq \sqrt{2 H^2 \log \left(\frac{\sqrt{1+k / \beta}}{\delta}\right) \frac{1}{\mathbf{C}_h^k(x, a)}}
$$

が任意の$(k,h)$と固定された$(x, a)$について，確率$1-\delta H$以上で成り立ちます．

最後に$(x,a)$について，Covering numberを使ってUnion boundを取りましょう．

$$
f_1(x, a) \stackrel{\text { def }}{=}\left|\frac{1}{\mathbf{C}_h^k(x, a)} \sum_{s=1}^{k-1} w_h^s(x, a) W_s\right| \text { and } f_2(x, a) \stackrel{\text { def }}{=} \sqrt{\frac{1}{\mathbf{C}_h^k(x, a)}}
$$

とすると，論文のLemma 8より，$\operatorname{Lip}\left(f_1\right) \leq 4 H C_2^g k /(\beta \sigma)$および$\operatorname{Lip}\left(f_2\right) \leq C_2^g k /\left(\beta^2 \sigma\right)$であることが言えます．

最後に，[RL_useful_lemma.ipynb](RL_useful_lemma.ipynb)の**補題：Covering Numberを使った２つの関数の大小の確率バウンド**を使えば，

$$
\begin{aligned}
\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) W_s\right| \leq & \sqrt{2 H^2 \log \left(\frac{\sqrt{1+k / \beta}}{\delta}\right) \frac{1}{\mathbf{C}_h^k(x, a)}} \\
& +\frac{\sigma^2}{K H} \operatorname{Lip}\left(f_1\right)+\frac{\sigma^2}{K H} \sqrt{2 H^2 \log \left(\frac{\sqrt{1+k / \beta}}{\delta}\right)} \operatorname{Lip}\left(f_2\right)
\end{aligned}
$$

が任意の$(x, a, k, h)$について，確率$1-\delta H \mathcal{N}\left(\sigma^2 /(K H), \mathcal{X} \times \mathcal{A}, \rho\right)$以上で成立することが言えます．

よって，

$$
\left|\widehat{P}_h^k V(x, a)-P_h V(x, a)\right| \leq\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) W_s\right|+2 \lambda_p L_1 \sigma\left(1+\sqrt{\log ^{+}\left(C_1^g k / \beta\right)}\right)+\frac{\beta H}{\mathbf{C}_h^k(x, a)}
$$

であることを使えば証明終わりです．


---

**証明**

$\widehat{P}_h^k(\cdot \mid x, a)$は今回はディラック測度の重み付き和担ってるので，$P_h(\cdot\mid x, a)$と$\widehat{P}_h^k(\cdot \mid x, a)$の距離をバウンドするのは難しいです．そこで，次の変形をします：

$$
\begin{aligned}
& \left|\left(\widehat{P}_h^k-P_h\right) V_{h+1}^*(x, a)\right| \\
& =\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) V_{h+1}^*\left(x_{h+1}^s\right)-P_h V_{h+1}^*(x, a)\right| \\
& \leq \underbrace{\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V_{h+1}^*\left(x_{h+1}^s\right)-P_h V_{h+1}^*\left(x_h^s, a_h^s\right)\right)\right|}_{(\mathbf{A})} \\
& +\underbrace{\lambda_p L_{h+1} \sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) \rho\left[(x, a),\left(x_h^s, a_h^s\right)\right]}_{(\mathbf{B})}+\underbrace{\frac{\beta\left\|V_{h+1}^*\right\|_{\infty}}{\mathbf{C}_h^k(x, a)}}_{(\mathbf{C})} .
\end{aligned}
$$

Aの部分はマルチンゲール差分列であり，特殊なHoeffdingでバウンドできます.
BはLipschitz性を利用したことによるバイアスです．
Cは$\beta$のせいで出てくるバウンドです．
ゴニョゴニョすると，結局

$$
\left|\left(\widehat{P}_h^k-P_h\right) V_{h+1}^*(x, a)\right| \lesssim \frac{H}{\sqrt{\mathbf{C}_h^k(x, a)}}+\frac{\beta H}{\mathbf{C}_h^k(x, a)}+L_1 \sigma .
$$

が出てきます．あとは通常のOptimismを使った証明とだいたい同じです（多分）．